## Machine Learning Model 

In [36]:
import psycopg2
import pandas as pd
import pandas.io.sql as sqlio
from config import database, user, password, host, port

con = psycopg2.connect(database=database, user=user, password=password, host=host, port=port)
sql = "SELECT * FROM Players;"
all_df = sqlio.read_sql_query(sql, con)
con = None

In [37]:
import warnings
warnings.filterwarnings('ignore')

In [38]:
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import LabelEncoder

#### Postgresql DB connection 
    we will display the dataframe now that we have connected to the postgresql with our notebook and we will display it through pandas.

In [39]:
all_df.head()

,name,age,year,team,league,player_salary,percent_season_played,war_psp,num_dl_movements,team_win_percentage,team_payroll,percent_contract_complete
0,Bobby Abreu,26,2000,PHI,NL,4354977,0.994676,6.213081,0,0.401235,69700927,33.333332
1,Bobby Abreu,27,2001,PHI,NL,7193330,1.029782,5.039903,0,0.530864,60144837,66.666664
2,Bobby Abreu,28,2002,PHI,NL,9000338,1.001989,5.818425,0,0.493827,82364470,100.000000
3,Bobby Abreu,29,2003,PHI,NL,12643905,1.016617,5.282226,0,0.530864,98344579,16.666668
4,Bobby Abreu,30,2004,PHI,NL,14346025,1.042947,6.289872,0,0.530864,126162692,33.333332


#### Columns and Target Variable
    We will create a list of the columns that are useful for our data as well as pick our target column - which is the WAR (Wins above Replacement) since that is our predictor variable.

In [28]:
columns = ["name", "age", "year", "team", "division", "league", "player_salary", "percent_season_played", "num_dl_movements", "team_win_percentage", 
           "team_payroll", "percent_contract_complete"]
          

target = ["war_psp"]


##### Variables to encode: team & league

    Loading in the data and cleaning it will be the next steps in this analysis. 

    We are first going to encode the categorical variables team and league in order to use them in our regression analysis.

    This will allow us to separate the data into testing and training sets. This is a resampling method. 

In [45]:
league_binary_encoded = pd.get_dummies(all_df, columns=["league"])
league_binary_encoded.head()

,name,age,year,team,player_salary,percent_season_played,war_psp,num_dl_movements,team_win_percentage,team_payroll,percent_contract_complete,league_AL,league_AL/NL,league_NL,league_NL/AL,league_NL/AL/AL,league_NL/AL/NL
0,Bobby Abreu,26,2000,PHI,4354977,0.994676,6.213081,0,0.401235,69700927,33.333332,0,0,1,0,0,0
1,Bobby Abreu,27,2001,PHI,7193330,1.029782,5.039903,0,0.530864,60144837,66.666664,0,0,1,0,0,0
2,Bobby Abreu,28,2002,PHI,9000338,1.001989,5.818425,0,0.493827,82364470,100.000000,0,0,1,0,0,0
3,Bobby Abreu,29,2003,PHI,12643905,1.016617,5.282226,0,0.530864,98344579,16.666668,0,0,1,0,0,0
4,Bobby Abreu,30,2004,PHI,14346025,1.042947,6.289872,0,0.530864,126162692,33.333332,0,0,1,0,0,0


In [57]:
league_binary_encoded.loc[(league_binary_encoded["league_NL/AL/NL"] == 1)]

,name,age,year,team,player_salary,percent_season_played,war_psp,num_dl_movements,team_win_percentage,team_payroll,percent_contract_complete,league_AL,league_AL/NL,league_NL,league_NL/AL,league_NL/AL/AL,league_NL/AL/NL
320,David DeJesus,33,2013,CHC/TBR/WSN,4664132,0.642151,2.958806,1,0.502058,100091461,66.666664,0,0,0,0,0,1


In [58]:
league_binary_encoded.loc[(league_binary_encoded["league_NL/AL/AL"] == 1)]

,name,age,year,team,player_salary,percent_season_played,war_psp,num_dl_movements,team_win_percentage,team_payroll,percent_contract_complete,league_AL,league_AL/NL,league_NL,league_NL/AL,league_NL/AL/AL,league_NL/AL/NL
1731,Jaime Garcia,30,2017,ATL/MIN/NYY,12515845,0.839572,1.465032,0,0.510288,142025719,100.0,0,0,0,0,1,0


    given that these two players are the only two players traded three times - it will take out points in the data but 2 out of our data set is not significant enough because of the size of the population 

In [59]:
all_df = all_df.loc[(all_df["name"] != "Jamie Garcia") | (all_df["name"] != "David DeJesus")]
all_df.head()

,name,age,year,team,league,player_salary,percent_season_played,war_psp,num_dl_movements,team_win_percentage,team_payroll,percent_contract_complete
0,Bobby Abreu,26,2000,PHI,NL,4354977,0.994676,6.213081,0,0.401235,69700927,33.333332
1,Bobby Abreu,27,2001,PHI,NL,7193330,1.029782,5.039903,0,0.530864,60144837,66.666664
2,Bobby Abreu,28,2002,PHI,NL,9000338,1.001989,5.818425,0,0.493827,82364470,100.000000
3,Bobby Abreu,29,2003,PHI,NL,12643905,1.016617,5.282226,0,0.530864,98344579,16.666668
4,Bobby Abreu,30,2004,PHI,NL,14346025,1.042947,6.289872,0,0.530864,126162692,33.333332


In [53]:
league_binary_encoded.loc[(league_binary_encoded["league_NL/AL"] == 1)].count()

name                         39
age                          39
year                         39
team                         39
player_salary                39
percent_season_played        39
war_psp                      39
num_dl_movements             39
team_win_percentage          39
team_payroll                 39
percent_contract_complete    39
league_AL                    39
league_AL/NL                 39
league_NL                    39
league_NL/AL                 39
league_NL/AL/AL              39
league_NL/AL/NL              39
dtype: int64

In [54]:
league_binary_encoded.loc[(league_binary_encoded["league_AL/NL"] == 1)].count()

name                         43
age                          43
year                         43
team                         43
player_salary                43
percent_season_played        43
war_psp                      43
num_dl_movements             43
team_win_percentage          43
team_payroll                 43
percent_contract_complete    43
league_AL                    43
league_AL/NL                 43
league_NL                    43
league_NL/AL                 43
league_NL/AL/AL              43
league_NL/AL/NL              43
dtype: int64

    To make the data more uniform - we are going to redefine all of the single trades in a season to be Uniform in AL/NL instead of both AL/NL and NL/AL 

In [67]:
all_df = all_df.replace(to_replace = "NL/AL", value="AL/NL", regex=True)



In [72]:
all_df.loc[(all_df["league"] == "NL/AL")]

,name,age,year,team,league,player_salary,percent_season_played,war_psp,num_dl_movements,team_win_percentage,team_payroll,percent_contract_complete


    above we have shown that the new all_df we will be working with has a uniform league column now - with no NL/AL values but with replaced AL/NL values. 
    
    Now we will create the encoded values for the different leagues 
    
    0 -> NL
    1 -> AL
    2 -> AL/NL
    
    and convert the values to the encoded values 

In [76]:
all_df = all_df.replace({'league': {'NL': 0, 'AL': 1, 'AL/NL': 2}})
all_df.head()

,name,age,year,team,league,player_salary,percent_season_played,war_psp,num_dl_movements,team_win_percentage,team_payroll,percent_contract_complete
0,Bobby Abreu,26,2000,PHI,0,4354977,0.994676,6.213081,0,0.401235,69700927,33.333332
1,Bobby Abreu,27,2001,PHI,0,7193330,1.029782,5.039903,0,0.530864,60144837,66.666664
2,Bobby Abreu,28,2002,PHI,0,9000338,1.001989,5.818425,0,0.493827,82364470,100.000000
3,Bobby Abreu,29,2003,PHI,0,12643905,1.016617,5.282226,0,0.530864,98344579,16.666668
4,Bobby Abreu,30,2004,PHI,0,14346025,1.042947,6.289872,0,0.530864,126162692,33.333332


Now we are going to view the different team names in this dataset:

In [90]:
team_names = all_df["team"].unique().tolist()
len(team_names)

169

    there are 169 unique team names / trades in our current dataframe
    
    from here we need to either encode the values or determine if the team name plays a imperative role in our analysis.

#### Now we will split our data into training and test sets 


In [ ]:
# Creating feautres

X = all_df.drop(columns="war_psp")
X = pd.get_dummies(X)

# Creating our target
y = MLB_df.loc[:, target].copy()

In [ ]:
X.describe()

In [ ]:
# check the balance of our target values 
y["war_psp"].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


### Method 1: Oversampling
comparing two oversampling algorithms, naive and SMOTE, will lead us to the best performing algorithm. 

The steps within both oversampling methods are as follows

1. View the count of the target classes using Counter from the collections library.
    1. Viewing the count of the target classes will allow us to see the official number of target variables that are used in this approach.
2. Use the resampled data to train a logistic regression model.
    1.  reamspling the training data to a logistic regression model is an attempt to have more accurate representation of points
3. Calculate the balanced accuracy score from sklearn.metrics.
    1. The balanced accuracy score will inform us on whether or not this method is statistically significant or not - if the accuracy is 95% or higher the model is considered to be an accurate representation. 
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the imbalanced_classification_report from imbalanced-learn.
